In [4]:
import numpy as np
import pandas as pd
import os

In [5]:
os.getcwd()

'/kaggle/working'

In [9]:
os.chdir("../input/m5-forecasting-accuracy")

In [61]:
INPUT_DIR = "../input/m5-forecasting-accuracy"
OUTPUT_DIR = "../../working/"

In [10]:
calendar = pd.read_csv("calendar.csv")

In [11]:
calendar

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0


In [12]:
!ls -GFlash

total 314M
4.0K drwxr-xr-x 2 nobody 4.0K Mar  3 00:35 ./
4.0K drwxr-xr-x 3 root   4.0K May 31 01:28 ../
104K -rw-r--r-- 1 nobody 102K Mar  3 00:35 calendar.csv
115M -rw-r--r-- 1 nobody 115M Mar  3 00:35 sales_train_validation.csv
5.0M -rw-r--r-- 1 nobody 5.0M Mar  3 00:35 sample_submission.csv
194M -rw-r--r-- 1 nobody 194M Mar  3 00:35 sell_prices.csv


In [13]:
!pwd

/kaggle/input/m5-forecasting-accuracy


In [32]:
def read_data():
    cal = pd.read_csv("calendar.csv")
    stv = pd.read_csv("sales_train_validation.csv")
    ss = pd.read_csv("sample_submission.csv")
    sellp = pd.read_csv("sell_prices.csv")
    
    return cal, stv, ss, sellp

In [33]:
cal, stv, ss, sellp = read_data()

In [20]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics: 
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [34]:
dfs = [cal, stv, ss, sellp]
for df in dfs:
    df = reduce_mem_usage(df)

Mem. usage decreased to  0.12 Mb (41.9% reduction)
Mem. usage decreased to 95.00 Mb (78.7% reduction)
Mem. usage decreased to  2.09 Mb (84.5% reduction)
Mem. usage decreased to 130.48 Mb (37.5% reduction)


In [36]:
sellp

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.578125
1,CA_1,HOBBIES_1_001,11326,9.578125
2,CA_1,HOBBIES_1_001,11327,8.257812
3,CA_1,HOBBIES_1_001,11328,8.257812
4,CA_1,HOBBIES_1_001,11329,8.257812
...,...,...,...,...
6841116,WI_3,FOODS_3_827,11617,1.000000
6841117,WI_3,FOODS_3_827,11618,1.000000
6841118,WI_3,FOODS_3_827,11619,1.000000
6841119,WI_3,FOODS_3_827,11620,1.000000


In [37]:
cal.fillna(0)

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,0,0,0,0,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,0,0,0,0,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,0,0,0,0,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,0,0,0,0,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,0,0,0,0,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,0,0,0,0,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,0,0,0,0,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,0,0,0,0,0,0,0


In [38]:
sellp.isnull().sum()

store_id      0
item_id       0
wm_yr_wk      0
sell_price    0
dtype: int64

In [41]:
stv.isnull().sum().max()

0

In [42]:
cal["event_name_1"].unique()

array([nan, 'SuperBowl', 'ValentinesDay', 'PresidentsDay', 'LentStart',
       'LentWeek2', 'StPatricksDay', 'Purim End', 'OrthodoxEaster',
       'Pesach End', 'Cinco De Mayo', "Mother's day", 'MemorialDay',
       'NBAFinalsStart', 'NBAFinalsEnd', "Father's day",
       'IndependenceDay', 'Ramadan starts', 'Eid al-Fitr', 'LaborDay',
       'ColumbusDay', 'Halloween', 'EidAlAdha', 'VeteransDay',
       'Thanksgiving', 'Christmas', 'Chanukah End', 'NewYear',
       'OrthodoxChristmas', 'MartinLutherKingDay', 'Easter'], dtype=object)

In [45]:
from sklearn.preprocessing import LabelEncoder

In [49]:
ev_list = ["event_name_1", "event_type_1", "event_name_2", "event_type_2"]
cal = pd.get_dummies(cal, columns=ev_list)

In [60]:
os.path.isdir("../../working")

True

In [64]:
cal.to_csv(OUTPUT_DIR + "cal_dummies.csv")